In [47]:
import matplotlib.pyplot as plt
import geopandas as gpd
import yaml
import os
import utility

In [48]:
config = utility.load_config()

def resolve_path(relative_path):
    return os.path.join(config['base_path'], relative_path)

Understanding how Africa_boundaries is chunked to evaluate how to overlay it with the CPIS shapefiles

In [49]:
Africa_boundaries = gpd.read_file(resolve_path(config['Africa_boundaries_shp_path']))

Africa_boundaries.loc[Africa_boundaries['Country']=="Cape Verde", 'Country'] = "Cabo Verde"

Africa_boundaries.to_file(resolve_path('Data/Raw/Africa_Boundaries-shp/Africa_Boundaries.shp'))

#cPIS2000 = gpd.read_file(resolve_path(config['CPIS_2000_shp_path']))
#cPIS2021 = gpd.read_file(resolve_path(config['CPIS_2021_shp_path']))

#Africa_boundaries.plot()
#plt.show()

This is how one would overlay the datasets to get the CPIS2000_Africa / CPIS2021_Africa sets (see below)... if their computer was beefier

In [50]:
#CPIS2000_Africa = gpd.overlay(cPIS2000, Africa_boundaries, how = 'intersection')
#CPIS2021_Africa = gpd.overlay(cPIS2021, Africa_boundaries, how = 'intersection')

#CPIS2000_Africa.plot()
#plt.show()

In [51]:
#CPIS2000_Africa['Year'] = 2000
#CPIS2021_Africa['Year'] = 2021

#CPIS2000_Africa['Area'] = CPIS2000_Africa.geometry.area
#CPIS2021_Africa['Area'] = CPIS2021_Africa.geometry.area

Replication of Map from Africa_corporate_irrigation (GitHub via anna-boser)

In [52]:
CPIS_Africa = gpd..read_file(resolve_path(config['Africa_CPIS_shp_path']))
CPIS2000_Africa = gpd.read_file(resolve_path(config['Africa_CPIS_2000_shp_path']))
CPIS2021_Africa = gpd.read_file(resolve_path(config['Africa_CPIS_2021_shp_path']))

# Buffer the geometry to make it more visible
CPIS_Africa['geometry'] = CPIS_Africa.geometry.buffer(0.1)

# Create a single plot with two facets
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot the CPIS data for 2000
Africa_boundaries.boundary.plot(ax=axes[0], color='black')
CPIS_Africa[CPIS_Africa['Year'] == 2000].plot(ax=axes[0], color='green')
axes[0].set_title('CPIS in 2000 in Africa')

# Plot the CPIS data for 2021
Africa_boundaries.boundary.plot(ax=axes[1], color='black')
CPIS_Africa[CPIS_Africa['Year'] == 2021].plot(ax=axes[1], color='green')
axes[1].set_title('CPIS in 2021 in Africa')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#Save the combined figure
#fig.savefig(utility.make_output_path('CPIS_2000_2021_Africa.png'))

SyntaxError: invalid syntax (2148753790.py, line 1)